
## Самостоятельная работа



In [1]:
import pandas as pd
import seaborn as sns
from scipy.stats import norm
import numpy as np
from sklearn import linear_model
from sklearn import metrics
from sklearn import ensemble
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler

%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

Будем работать с датасетом Pima Indian Diabetes - это набор данных из Национального института диабета, болезней органов пищеварения и почек. Целью набора данных является диагностическое прогнозирование наличия диабета у пациента. Несколько ограничений были наложены на выбор этих экземпляров из большой базы данных. В частности, все пациенты здесь - женщины в возрасте от 21 года, индийского происхождения.

In [2]:
data = pd.read_csv('data_demo.csv')
data.head(10)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Class
0,6,148.0,72.0,35.0,NaN,33.6,0.627,50,1
1,1,85.0,66.0,29.0,NaN,26.6,0.351,31,0
2,8,183.0,64.0,NaN,NaN,23.3,0.672,32,1
3,1,89.0,66.0,23.0,94.0,28.1,0.167,21,0
4,0,137.0,40.0,35.0,168.0,43.1,2.288,33,1
5,5,116.0,74.0,NaN,NaN,25.6,0.201,30,0
6,3,78.0,50.0,32.0,88.0,31.0,0.248,26,1
7,10,115.0,NaN,NaN,NaN,35.3,0.134,29,0
8,2,197.0,70.0,45.0,543.0,30.5,0.158,53,1
9,8,125.0,96.0,NaN,NaN,NaN,0.232,54,1


Описание данных:

- __Pregnancies__ - данная единица отображает количество беременностей, единицы измерения - целые числа от 0 до N. Тип переменной - количественная, дискретная.
- __Glucose__ - данная единица отображает уровень глюкозы в крови, единицы измерения - целые числа. Тип переменной - количественная, дискретная.
- __BloodPressure__ - данная единица отображает артериальное давление, единицы измерения - миллиметры р/с, целые числа. Тип переменной - количественная, дискретная.
- __SkinThickness__ - данная единица отображает обхват трицепса в миллиметрах, целые числа. Тип переменной - количественная, дискретная.
- __Insulin__ - данная единица отображает уровень инсулина в крови, целые числа. Тип переменной - количественная, дискретная.
- __BMI__ - данная единица отображает индекс массы тела. Тип переменной - количественная, непрерывная.
- __DiabetesPedigreeFunction__ - данная единица отображает риск наследственного диабета в зависимости наличия диабета у родственников. Выражается десятичной дробью от 0 до 1. Тип переменной - количественная, непрерывная.
- __Age__ - данная единица отражает возраст в целых числах. Тип переменной - количественная, дискретная.
- __Class__ - данная единица отражает наличие диабета у субъекта, выражена 0(здоров) или 1(болен). Тип переменной - категориальная, бинарная.

__Задание 1. (1 балл)__

Как вы видите, в данных много пропусков (NaN). Любым способом посчитайте количество пропусков в каждом из столбцов (но должно быть понятно, как вы это делали).

In [3]:
data.isna().sum()

Pregnancies                   0
Glucose                       5
BloodPressure                35
SkinThickness               227
Insulin                     374
BMI                          11
DiabetesPedigreeFunction      0
Age                           0
Class                         0
dtype: int64

__Задание 2. (1 балл)__

Замените все пропуски модами, медианами или средними (выберите сами)

In [4]:
data.Glucose = data.Glucose.fillna(data.Glucose.median())
data.BloodPressure = data['BloodPressure'].fillna(data['BloodPressure'].median())
data.SkinThickness = data.SkinThickness.fillna(data.SkinThickness.median())
data.Insulin = data['Insulin'].fillna(data['Insulin'].median())
data.BMI = data['BMI'].fillna(data['BMI'].mean())

In [5]:
data.isna().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Class                       0
dtype: int64

__Задание 3. (1 балл)__

Будем предсказывать наличие или отсутствие диабета. Разделите данные на обучающую и тестовую выборку в соотношении 3 к 1.

In [6]:
features = data.drop('Class', axis=1).columns

X, y = data[features], data['Class']

In [7]:
from sklearn.model_selection import train_test_split

y_train, y_test, X_train, X_test = train_test_split(y, X, test_size = 0.25, random_state = 42)

__Задание 4. (2 балла)__

 Постройте наивную модель предсказания для целевой переменной. Наивная модель - это самая простая модель, без сложных зависимостей. Например, для регрессии наивная модель - среднее для целевой переменной. В данном случае наивной моделью может быть преположение "ни у кого нет диабета". Оцените качество модели.

In [8]:
naive_prediction = np.zeros(X_test.shape[0])

accuracy = accuracy_score(y_test, naive_prediction)
print('Accuracy наивной модели:', accuracy)

Accuracy наивной модели: 0.640625


__Задание 5. (3 балла)__

Постройте предсказательную модель с помощью логистической регрессии и случайного леса, а также KNN c 5 соседями. Сравните качество.

In [9]:
log_reg = linear_model.LogisticRegression(
    solver='sag',
    random_state=1,
    max_iter=1000
)

log_reg.fit(X_train, y_train)

LogisticRegression(max_iter=1000, random_state=1, solver='sag')

In [10]:
y_train_pred = log_reg.predict(X_train)
print(metrics.classification_report(y_train, y_train_pred))

y_test_pred = log_reg.predict(X_test)
print(metrics.classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       0.73      0.85      0.79       377
           1       0.59      0.41      0.49       199

    accuracy                           0.70       576
   macro avg       0.66      0.63      0.64       576
weighted avg       0.68      0.70      0.68       576

              precision    recall  f1-score   support

           0       0.73      0.85      0.78       123
           1       0.62      0.45      0.52        69

    accuracy                           0.70       192
   macro avg       0.68      0.65      0.65       192
weighted avg       0.69      0.70      0.69       192



In [11]:
rf = ensemble.RandomForestClassifier(
    n_estimators=500,
    max_depth=15,
    criterion='entropy',
    min_samples_leaf=10,
    random_state=42
)

rf.fit(X_train, y_train)

y_train_pred = rf.predict(X_train)
print('Train: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
y_test_pred = rf.predict(X_test)
print('Test: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

Train: 0.78
Test: 0.64


In [12]:
knn = KNeighborsClassifier(n_neighbors = 5)

knn.fit(X_train, y_train)

y_pred_knn = knn.predict(X_test.values)

p_pred_knn = knn.predict_proba(X_test.values)

accuracy_score(y_test, y_pred_knn)

0.6458333333333334

#### Задание 6 (2 балла)

Примените шкалирование (MinMax, обучите его на обучающей выборке для признаков, затем примените к тестовой), снова постройте модели.

In [13]:
min_max_scaler = MinMaxScaler()
min_max_scaler.fit(X_train)

X_scaled_train = min_max_scaler.transform(X_train)
X_scaled_test = min_max_scaler.transform(X_test)

In [14]:
log_reg = linear_model.LogisticRegression(
    solver='sag',
    random_state=1,
    max_iter=1000
)

log_reg.fit(X_scaled_train, y_train)

y_train_pred_scaled = log_reg.predict(X_scaled_train)
print(metrics.classification_report(y_train, y_train_pred))

y_test_pred_scaled = log_reg.predict(X_scaled_test)
print(metrics.classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       0.87      0.92      0.89       377
           1       0.83      0.73      0.78       199

    accuracy                           0.86       576
   macro avg       0.85      0.83      0.84       576
weighted avg       0.85      0.86      0.85       576

              precision    recall  f1-score   support

           0       0.79      0.83      0.81       123
           1       0.67      0.61      0.64        69

    accuracy                           0.75       192
   macro avg       0.73      0.72      0.72       192
weighted avg       0.75      0.75      0.75       192



In [15]:
rf = ensemble.RandomForestClassifier(
    n_estimators=500,
    max_depth=15,
    criterion='entropy',
    min_samples_leaf=10,
    random_state=42
)

rf.fit(X_scaled_train, y_train)

y_train_pred = rf.predict(X_scaled_train)
print('Train: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
y_test_pred = rf.predict(X_scaled_test)
print('Test: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

Train: 0.78
Test: 0.64


In [16]:
knn = KNeighborsClassifier(n_neighbors = 5)

knn.fit(X_scaled_train, y_train)

y_pred_knn = knn.predict(X_scaled_test)

p_pred_knn = knn.predict_proba(X_scaled_test)

accuracy_score(y_test, y_pred_knn)

0.6979166666666666